## MF Portfolio Visulizer

### TODO
 - [x] Connect ICRA APIs to pull portfolio holdings `class:mfAssetAllocations()`
 - [x] Visualise MF portfolio by individual assets and asset type
  - [x] Add switch of visuals on MF portfolios and personal investments `weighted=True`
 - [x] Add visuals to see changes in current holdings on new fund addition
 - [ ] Connect APIs to track individual asset growth
 - [ ] Add historical NAVs and visuals for fund returns

In [ ]:
import sys
import os.path
import bs4
import requests
import pandas as pd
from bs4 import BeautifulSoup
from functools import partial, reduce
from collections import defaultdict

# sys.path.insert(0, '/home/j5067250/credentials')
# from credentials import HTTP_PROXY, HTTPS_PROXY

%matplotlib inline
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
%load_ext watermark
%watermark --iversion

requests  : 2.23.0
bs4       : 0.0.1
plotly    : 5.4.0
IPython   : 5.5.0
matplotlib: 3.2.2
pandas    : 1.1.5
sys       : 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]



In [ ]:
import plotly.io as pio
pio.templates

Templates configuration
-----------------------
    Default template: 'plotly'
    Available templates:
        ['ggplot2', 'seaborn', 'simple_white', 'plotly',
         'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
         'ygridoff', 'gridon', 'none']

In [ ]:
class mfAssetAllocations():
    """class `mfAssetAllocation`
    
    Reads asset allocation (portfolio) of a MF,
    and returns a pandas dataframe
    """
    def __init__(self, name, url, refresh=False, batch_size=3):
        """Initialising `class mfAssetAllocation`
        
        Inputs:
            name(string): name of the mutual fund
            url(string): MF Portfolio webpage on ICRA Website
            refresh(bool): reload portfolio values from webpage (refresh=True) 
                or read from the pickle dataframe
            batch_size(int): Number of columns expected from HTML table
        """
        self.name = name
        self.url = url
        self.refresh = refresh
        self.BATCH_SIZE = batch_size
    
    def __call__(self):
        if self.refresh is True or not os.path.isfile('./Data/' + self.name + '.pkl') :
            print(f'Downloading Portfolio for {self.name}')
            return self.pipeline()
        else:
            print("Loading saved pickle file")
            return pd.read_pickle('./Data/' + self.name + '.pkl')
    
    def pipeline(self):
        """Pipeline to read webpage and convert to dataframe
        """
        self.html_text = self.read_html()
        self.beautiful_text = self.beautify_html()
        self.extracted_data_list = self.extract_data()
        self.processed_df = self.process_data()
        self.processed_df.to_pickle('./Data/' + self.name + '.pkl')
        return self.processed_df
        
    def read_html(self):
        """Load webpage using `requests` module
        """
        self.response = requests.get(
        url=self.url)
        self.html_doc = self.response.text
        
    def beautify_html(self):
        """Refine HTML text from portfolio page
        """
        self.soup = BeautifulSoup(self.html_doc, 'html.parser')
        self.item = self.soup.find_all('div', {"class": "accordion-body collapse in"})[0]
        self.html_text = self.item.text.strip()
        return self.html_text
    
    def extract_data(self):
        """Extract portfolio company & asset details from text
        """
        self.texts = [text for text in self.html_text.split('\n') if text and text != 'Detailed Portfolio']
        texts_subset = self.texts[self.BATCH_SIZE:]
        self.data = defaultdict(list)
        while texts_subset:
            for i in range(self.BATCH_SIZE):
                text = texts_subset.pop()
                if i == 0:
                    self.data['Percentage Allocation'].append(text)
                elif i == 1:
                    self.data['Asset Type'].append(text)
                elif i == 2:
                    self.data['Company Name'].append(text)
        return self.data
    
    def process_data(self):
        """Convert extracted data to dataframe
        """
        df = pd.DataFrame(self.extracted_data_list)
        df['Asset Type'] = df['Asset Type'].str.strip()
        df['Percentage Allocation'] = df['Percentage Allocation'].str.rstrip(' %')
        df['Percentage Allocation'] = df['Percentage Allocation'].astype('float')
        df = df.groupby(['Company Name', 'Asset Type'])['Percentage Allocation'].agg('sum').reset_index()
        return df

In [ ]:
ref = {
    'Axis Focused 25 Fund': {
        'url': 'https://www.mutualfundindia.com/MF/Portfolio/Details/16299',
        'amount': 163735
    },
    'Parag Parikh Flexi Cap Fund' : {
        'url': 'https://www.mutualfundindia.com/MF/Portfolio/Details/20004',
        'amount': 53233
    },
    'Kotak Equity Hybrid Fund': {
        'url': 'https://www.mutualfundindia.com/MF/Portfolio/Details/17288',
        'amount': 102263
    },
    'IDFC Focused Equity Fund': {
        'url': 'https://www.mutualfundindia.com/MF/Portfolio/Details/18094',
        'amount': 59132
    },
    'Axis Long Term Equity Fund': {
        'url': 'https://www.mutualfundindia.com/MF/Portfolio/Details/16271',
        'amount': 149376
    },
    'DSP Tax Saver Fund': {
        'url': 'https://www.mutualfundindia.com/MF/Portfolio/Details/16819',
        'amount': 58677
    },
    'Invesco India Tax Plan Fund': {
        'url': 'https://www.mutualfundindia.com/MF/Portfolio/Details/17484',
        'amount': 57620
    },
    'Kotak Tax Saver Fund': {
        'url': 'https://www.mutualfundindia.com/MF/Portfolio/Details/17355',
        'amount': 57302
    },
    'ICICI Prudential Ultra Short Term Fund': {
        'url': 'https://www.mutualfundindia.com/MF/Portfolio/Details/17114',
        'amount': 419043
    }
}

In [ ]:
def overall_portfolio(ref=ref, refresh=False):
    """Calculates investments in each asset (company) from reference investment amounts
    
    Input:
        ref(dict): References data with investment amount and portfolio weblinks
        
    Return:
        DataFrame: Dataframe with overall investments in each asset (company)
    """
    portfolio_df = pd.DataFrame(columns=['Company Name', 'Asset Type', 'Weighted Amt'])
    for mf in ref:
        weights_df = mfAssetAllocations(name=mf, url=ref[mf]['url'], refresh=refresh)()
        total_investment = ref[mf]['amount']
        weights_df['Weighted Amt'] = weights_df['Percentage Allocation'] * total_investment / 100
        weights_df = weights_df.drop(columns=['Percentage Allocation'], axis=1)
        
        portfolio_df = portfolio_df.merge(weights_df,
                                          how='outer',
                                          on=['Company Name', 'Asset Type'])
        portfolio_df = portfolio_df.fillna(0)
        portfolio_df['Weighted Amt'] = portfolio_df['Weighted Amt_x'] + portfolio_df['Weighted Amt_y']
        portfolio_df = portfolio_df.drop(columns=['Weighted Amt_x', 'Weighted Amt_y'])
        
    return portfolio_df

In [ ]:
my_investments = overall_portfolio(refresh=True)

In [ ]:
pd.set_option('display.max_rows', 10)
my_investments.sort_values(by='Weighted Amt')

,Company Name,Asset Type,Weighted Amt
41,Currency Derivatives,Equity,-12493.7851
19,Net Receivables/(Payable),Others,-10179.5938
107,Nifty,Equity,-2035.0337
264,Axis Bank Ltd.,Debt,-1759.9806
75,8.19 Rajasthan UDAY Bond Jun 23 2026,Debt,0.0000
...,...,...,...
27,Tata Consultancy Services Ltd.,Equity,34038.5827
14,ICICI Bank Ltd.,Equity,34183.3741
5,Bajaj Finance Ltd.,Equity,35864.4812
306,Treasury Bill (182D),Others,45759.4956


In [ ]:
my_investments['Weighted Amt'].sum()

1120571.638

In [ ]:
tracked_amt = 0
actual_amt = 0
for mf in ref:
    _df = mfAssetAllocations(name=mf, url=ref[mf]['url'])()
    pc = _df['Percentage Allocation'].sum()
    tracked_amt += ref[mf]['amount'] * pc / 100
    actual_amt += ref[mf]['amount']

print("Tracked Amount: ", tracked_amt)
print("Investment Value: ", actual_amt)
print("Difference: ", abs(tracked_amt - actual_amt))

Loading saved pickle file
Loading saved pickle file
Loading saved pickle file
Loading saved pickle file
Loading saved pickle file
Loading saved pickle file
Loading saved pickle file
Loading saved pickle file
Loading saved pickle file
Tracked Amount:  1120571.638
Investment Value:  1120381
Difference:  190.6380000000354


In [ ]:
def fund_holdings(ref, weighted=False):
    """Proportion of investment of each fund in each asset.
    If weighted is True, returns actual invesment amount for personal invesments from ref.
    
    Input:
        ref(dict): References data with investment amount and portfolio weblinks
        weighted(bool): If True, each fund column has actual investment proportion,
            else a portfolio percentage allocations.
    """
    dfs = [pd.read_pickle('./Data/' + mf + '.pkl').rename(columns={'Percentage Allocation': mf}) for mf in ref]
    if weighted is True:
        for (mf, df) in zip(ref, dfs):
            df[mf] = df[mf] * ref[mf]['amount'] / 100
    df = reduce(partial(pd.core.frame.DataFrame.merge, how='outer', on=['Company Name', 'Asset Type']), dfs).fillna(0)
    df['Total Holdings'] = df[list(ref.keys())].sum(axis=1)
    return df

In [ ]:
def plot_companies(df, ref, top_n=25, viz='plotly'):
    """Plot top N companies 
    
    Input:
        df(DataFrame): Holdings dataframe with a column for each fund holding value
        ref(Dictionary): References
        top_n(Integer): Number of assets in the plot with top holdings
        viz(String): Visualisation library to use (plotly or matplotlib)
    """
    
    if top_n is None:
        top_n = df.shape[0]
    
    funds = list(ref.keys())
    title = 'Holdings distribution in each asset'
    
    index = df['Total Holdings'].sort_values(ascending=False).index
    df = df.iloc[index[:top_n]]
    
    if viz == 'matplotlib':
        df.plot.bar(
            x='Company Name',
            y=funds,
            label=funds,
            stacked=True,
            title=title,
            rot=90)
    elif viz == 'plotly':
        fig = px.bar(
                df,
                x='Company Name',
                y=funds,
                title=title,
                template='plotly'
                )
        fig.update_layout(
            font=dict(
                size=10),
            width=df.shape[0]*30,
            height=600
        )
        fig.update_xaxes(title_font_family="Arial")
        fig.show()

In [ ]:
df_holdings = fund_holdings(ref, weighted=False)
plot_companies(df_holdings, ref, top_n=30)

In [ ]:
df_holdings.head()

,Company Name,Asset Type,Axis Focused 25 Fund,Parag Parikh Flexi Cap Fund,Kotak Equity Hybrid Fund,IDFC Focused Equity Fund,Total Holdings
0,Avenue Supermarts Ltd.,Equity,9.77,0.0,0.0,0.0,9.77
1,Axis Bank Ltd. (368D),Debt,0.12,0.0,0.0,0.0,0.12
2,Axis Bank Ltd. (370D),Debt,0.37,0.0,0.0,0.0,0.37
3,Axis Bank Ltd. (372D),Debt,1.36,0.0,0.0,0.0,1.36
4,Axis Bank Ltd. (92D),Debt,0.12,0.0,0.0,0.0,0.12


In [ ]:
def plot_assets(df, ref, viz='plotly'):
    """Plot holdings by asset type
    """
    funds = list(ref.keys())
    title = "Holdings Distribution by Asset Type"
    
    df = df.groupby('Asset Type')['Total Holdings'].agg('sum').reset_index()
    
    total = df['Total Holdings'].sum()
    labels = df['Asset Type'] + ', ' + (df['Total Holdings']/total*100).round(2).astype(str) + '%'
    
    if viz == 'matplotlib':
        df.plot.pie(
            y='Total Holdings',
            labels=labels
        )
    elif viz == 'plotly':
        fig = px.pie(
                df,
                names='Asset Type',
                values='Total Holdings',
                hole=0.5,
                title=title,
            )
        fig.update_layout(
            font=dict(
                size=10),
            width=400,
            height=400
        )
        fig.update_xaxes(title_font_family="Arial")
        fig.show()

In [ ]:
plot_assets(df_holdings, ref)

In [ ]:
def plot_correlation(ref, fund1, fund2, top_n=None, sort_by='corr'):
    """Function to plot holdings of 2 funds showing the correlation in their portfolio assets
        
    Input:
        top_n(Integer): Number of high value holdings to plot
        sort_by(String): Sort final dataframe either by Correlation % or Absolute Holdings (hol)
    """
    
    title = f"Fund Portfolio Correlation of {fund1} v/s {fund2}"
    
    df = fund_holdings(ref, weighted=False)
    df = df[['Company Name', 'Asset Type', fund1, fund2]]
    df[fund1] = abs(df[fund1])
    df[fund2] = abs(df[fund2])
    df['Total Holdings'] = df[[fund1, fund2]].sum(axis=1)
    df = df[df['Total Holdings'] > 0]
    
    df['Corr'] = df[[fund1, fund2]].min(axis=1)
    
    if sort_by == 'corr':
        df = df.sort_values(by='Corr', ascending=False)
    elif sort_by == 'hol':
        df = df.sort_values(by='Total Holdings', ascending=False)
    
    if top_n is not None:
        df = df.iloc[:top_n,:]

    corr_trace = go.Bar(
        name='Correlation',
        x=df['Company Name'],
        y=df['Corr'])
    
    fund1_trace = go.Bar(
        name=fund1,
        x=df['Company Name'],
        y=df[fund1])
    
    fund2_trace = go.Bar(
        name=fund2,
        x=df['Company Name'],
        y=df[fund2])
    
    fig = make_subplots(rows=2, shared_xaxes=True, horizontal_spacing=0.8, row_heights=[40, 10])
    fig.add_trace(corr_trace, row=2, col=1)
    
    fig.add_trace(fund1_trace, row=1, col=1)
    fig.add_trace(fund2_trace, row=1, col=1)
    fig.update_layout(
        barmode='stack',
        title=title,
        xaxis_title='Company Name',
        font=dict(
            size=10),
        width=df.shape[0]*40,
        height=600)
    
    print(f"\nIf invested equal amount in both \n{fund1} and {fund2}, \n{round(df['Corr'].sum(), 2)}% amount from each fund would be invested in same assets.")
    fig.show()

In [ ]:
funds = list(ref.keys())

for i in range(len(funds)):
    for j in range(i+1, len(funds)):
        plot_correlation(ref, funds[i], funds[j], top_n=25, sort_by='corr')


If invested equal amount in both 
Axis Focused 25 Fund and Parag Parikh Flexi Cap Fund, 
6.92% amount from each fund would be invested in same assets.



If invested equal amount in both 
Axis Focused 25 Fund and Kotak Equity Hybrid Fund, 
20.02% amount from each fund would be invested in same assets.



If invested equal amount in both 
Axis Focused 25 Fund and IDFC Focused Equity Fund, 
26.1% amount from each fund would be invested in same assets.



If invested equal amount in both 
Axis Focused 25 Fund and Axis Long Term Equity Fund, 
78.2% amount from each fund would be invested in same assets.



If invested equal amount in both 
Axis Focused 25 Fund and DSP Tax Saver Fund, 
12.37% amount from each fund would be invested in same assets.


...


If invested equal amount in both 
DSP Tax Saver Fund and Kotak Tax Saver Fund, 
41.53% amount from each fund would be invested in same assets.



If invested equal amount in both 
DSP Tax Saver Fund and ICICI Prudential Ultra Short Term Fund, 
1.67% amount from each fund would be invested in same assets.



If invested equal amount in both 
Invesco India Tax Plan Fund and Kotak Tax Saver Fund, 
33.51% amount from each fund would be invested in same assets.



If invested equal amount in both 
Invesco India Tax Plan Fund and ICICI Prudential Ultra Short Term Fund, 
1.78% amount from each fund would be invested in same assets.



If invested equal amount in both 
Kotak Tax Saver Fund and ICICI Prudential Ultra Short Term Fund, 
2.52% amount from each fund would be invested in same assets.


### Compare current folio funds to new fund

In [ ]:
def plot_new_fund(ref, current_mfs, new_mf, sort_by='hol', top_n=None):
    
    if new_mf in current_mfs:
        raise ValueError("The index of new fund must not be in current funds.")
    
    current_mfs = [mf for (mf, idx) in zip(ref, range(len(ref))) if idx in current_mfs]
    new_mf = [mf for (mf, idx) in zip(ref, range(len(ref))) if idx == new_mf][0]
    
    dfs = [pd.read_pickle('./Data/' + mf + '.pkl').rename(columns={'Percentage Allocation': mf}) for mf in current_mfs]
    df_holdings = reduce(partial(pd.core.frame.DataFrame.merge, how='outer', on=['Company Name', 'Asset Type']), dfs).fillna(0)
    df_holdings['Current Holdings'] = df_holdings[current_mfs].sum(axis=1)
    df_holdings = df_holdings.drop(columns=current_mfs)
    
    df_new = pd.read_pickle('./Data/' + new_mf + '.pkl').rename(columns={'Percentage Allocation': new_mf})
    df = df_new.merge(df_holdings, on=['Company Name', 'Asset Type'], how='inner').fillna(0)
    
    ## Visualisation 
    title = f"Fund Portfolio Correlation of current investments v/s {new_mf}"
    
    df[new_mf] = abs(df[new_mf])
    df['Current Holdings'] = abs(df['Current Holdings'])
    df['Total Holdings'] = df[['Current Holdings', new_mf]].sum(axis=1)
    df = df[df['Total Holdings'] > 0]
    
    df['Corr'] = df[['Current Holdings', new_mf]].min(axis=1)
    
    if sort_by == 'corr':
        df = df.sort_values(by='Corr', ascending=False)
    elif sort_by == 'hol':
        df = df.sort_values(by='Total Holdings', ascending=False)
    
    if top_n is not None:
        df = df.iloc[:top_n,:]

    corr_trace = go.Bar(
        name='Correlation',
        x=df['Company Name'],
        y=df['Corr'],
        marker_color='steelblue',
        width=0.5)
    
    hol_trace = go.Bar(
        name='Current Holdings',
        x=df['Company Name'],
        y=df['Current Holdings'],
        marker_color='grey')
    
    fund2_trace = go.Bar(
        name=new_mf,
        x=df['Company Name'],
        y=df[new_mf],
        marker_color='firebrick')
    
    fig = make_subplots(rows=2, shared_xaxes=True, horizontal_spacing=0.8, row_heights=[40, 10])
    fig.add_trace(corr_trace, row=2, col=1)
    
    fig.add_trace(hol_trace, row=1, col=1)
    fig.add_trace(fund2_trace, row=1, col=1)
    fig.update_layout(
        barmode='stack',
        title=title,
        xaxis_title='Company Name',
        font=dict(
            size=10),
        width=df.shape[0]*40,
        height=600)
    
    print(f"\nYour current portfolio shares {round(df['Corr'].sum(), 2)}% holdings with {new_mf}.")
    fig.show()


In [ ]:
for i in range(len(ref)):
    new_fund = i
    current_fund_indexes_in_ref = [idx for idx in range(len(ref)) if idx != new_fund]
    plot_new_fund(ref, current_fund_indexes_in_ref, new_fund)


Your current portfolio shares 36.84% holdings with Axis Focused 25 Fund.



Your current portfolio shares 22.94% holdings with Parag Parikh Flexi Cap Fund.



Your current portfolio shares 47.1% holdings with Kotak Equity Hybrid Fund.



Your current portfolio shares 56.18% holdings with IDFC Focused Equity Fund.
